<a href="https://colab.research.google.com/github/sarthakc123/Sarthak_Vinit_ClimateRiskMitigationMCSim/blob/main/MCSim_ClimateRisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson, nbinom
from scipy.stats import gamma



In [ ]:
df_New_Orleans = pd.read_csv('New_Orleans.csv')

df_New_Orleans.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,198911,15,1607,198911,15,1607,NaN,10045593,LOUISIANA,22.0,...,0.0,NaN,NaN,30.05,-90.03,NaN,NaN,NaN,NaN,PUB
1,198905,4,1750,198905,4,1750,NaN,10046667,LOUISIANA,22.0,...,0.0,NaN,NaN,30.05,-90.70,NaN,NaN,NaN,NaN,PUB
2,198905,4,1800,198905,4,1800,NaN,10046669,LOUISIANA,22.0,...,0.0,NaN,NaN,30.00,-90.48,NaN,NaN,NaN,NaN,PUB
3,198905,4,1815,198905,4,1815,NaN,10046670,LOUISIANA,22.0,...,0.0,NaN,NaN,30.00,-90.25,NaN,NaN,NaN,NaN,PUB
4,198905,4,1830,198905,4,1830,NaN,10046671,LOUISIANA,22.0,...,0.0,NaN,NaN,29.95,-90.10,NaN,NaN,NaN,NaN,PUB


In [ ]:
df_New_Orleans['DAMAGE_PROPERTY'].sum()

np.float64(35468073000.0)

In [ ]:
df_New_Orleans = df_New_Orleans[(df_New_Orleans['YEAR'] >= 1994) & (df_New_Orleans['YEAR'] <= 2024)]

In [ ]:
df_New_Orleans.shape

(1191, 51)

# FUNCTION FOR FLOOD FREQUENCY IN A YEAR


In [ ]:
# Count number of flood events per year
flood_frequency = df_New_Orleans.groupby('YEAR').size()

# Convert flood_frequency Series to DataFrame
df_floods = flood_frequency.reset_index()
df_floods.columns = ['Year', 'Floods']

# Sort by year just to be sure
df_floods = df_floods.sort_values('Year')

def simulate_flood_frequency(df_floods, n_years=None, seed=None):
    if seed is not None:
        np.random.seed(seed)

    mu = df_floods["Floods"].mean()
    var = df_floods["Floods"].var()

    if var > mu:
        p_nb = mu / var
        r_nb = mu**2 / (var - mu)
        dist = nbinom(r_nb, p_nb)
    else:
        dist = poisson(mu)

    if n_years is None:
        n_years = df_floods.shape[0]

    simulated_floods = dist.rvs(size=n_years)

    # Clamp to min=1 and max=mu + 7*σ
    lower = 1
    upper = int(mu + 7 * np.sqrt(var))
    return np.clip(simulated_floods, lower, upper)

simulated_counts = simulate_flood_frequency(df_floods, n_years=30)
print(simulated_counts)

[ 39  40 123  37  26  81  34  55  54  58  45  43  41  44  44  27  62  17
  34  60  30  68  17  26  57  42  34 119  61  20]


In [ ]:
def assign_severity(df):
    # Define thresholds using quantiles
    q1 = 1_000_000
    q2 = 10_000_000
    q3 = 100_000_000

    def categorize(damage):
        if damage <= q1:
            return 'Low'
        elif damage <= q2:
            return 'Medium'
        elif damage <= q3:
            return 'High'
        else:
            return 'Extreme'

    df['SEVERITY_LEVEL'] = df['DAMAGE_PROPERTY'].apply(categorize)
    return df


df_New_Orleans = assign_severity(df_New_Orleans)

df_New_Orleans['SEVERITY_LEVEL'].value_counts()


,count
SEVERITY_LEVEL,
Low,1093
Medium,62
High,22
Extreme,14


In [ ]:
# Define severity counts (from your data)
severity_counts = {
    "Low": 1093,
    "Medium": 62,
    "High": 22,
    "Extreme": 14
}

# Calculate total and probabilities
total = sum(severity_counts.values())
severity_levels = list(severity_counts.keys())
probabilities = [count / total for count in severity_counts.values()]

# Function to generate random severity values based on distribution
def generate_random_severity(n=1):
    return np.random.choice(severity_levels, size=n, p=probabilities)

# Example usage
print(generate_random_severity(30))  # Generate 10 random severities

['Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Medium' 'Low' 'Low' 'Low'
 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low'
 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low']


In [ ]:
df = pd.read_csv("/content/SEA LEVEL RISE.csv")
#df_seadata.head()

df_louisiana_slr = df[
    (df['Latitude'] >= 28.5) & (df['Latitude'] <= 33.0) &
    ((df['Longitude'] <= -88.8) & (df['Longitude'] >= -94.0) |
     (df['Longitude'] >= 266.0) & (df['Longitude'] <= 271.2))  # For 0–360 longitude
]

columns_to_keep = [
    "Site",
    "PSMSL ID",
    "Latitude",
    "Longitude",
    "Scenario",
    "Background RSL rate (mm/yr)",
    "RSL in 2000 (cm)",
    "RSL in 2010 (cm)",
    "RSL in 2020 (cm)",
    "RSL in 2030 (cm)"
]

df_louisiana_slr = df_louisiana_slr[columns_to_keep]
#df_louisiana_slr["Site"].unique()
df_louisiana_slr=df_louisiana_slr.loc[df_louisiana_slr["Site"] == "GRAND ISLE"]
df_louisiana_slr.head()

# Filter for Grand Isle, 0.5 - MED scenario
row = df_louisiana_slr[df_louisiana_slr["Scenario"] == "0.5 - MED"].iloc[0]

# Step 1: Extract years and known SLR values
years_known = np.array([2000, 2010, 2020, 2030])
slr_known = np.array([
    row["RSL in 2000 (cm)"],
    row["RSL in 2010 (cm)"],
    row["RSL in 2020 (cm)"],
    row["RSL in 2030 (cm)"]
])

# Step 2: Predict for 2025–2054 using linear interpolation + extrapolation
years_future = np.arange(2025, 2055)
# Fit linear model
coeffs = np.polyfit(years_known, slr_known, deg=1)
slope, intercept = coeffs
slr_predicted = slope * years_future + intercept

In [ ]:
def generate_random_slr(n=1):
    return np.random.choice(slr_predicted, size=n)

generate_random_slr(30)

array([42.99, 28.95, 34.8 , 55.86, 50.01, 46.5 , 37.14, 60.54, 28.95,
       59.37, 59.37, 46.5 , 38.31, 42.99, 40.65, 42.99, 57.03, 52.35,
       39.48, 32.46, 59.37, 34.8 , 60.54, 35.97, 33.63, 40.65, 42.99,
       58.2 , 57.03, 50.01])

In [ ]:
# Load and filter dataset
df = pd.read_csv("Precipitaton.csv")
prcp_values = df["PRCP"].dropna()

# Fit Gamma distribution to historical precipitation data
shape, loc, scale = gamma.fit(prcp_values)

# Function to generate n simulated precipitation values
def generate_random_precipitation(n=1, random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)
    return gamma.rvs(shape, loc=loc, scale=scale, size=n)



In [ ]:
alpha = 1000000
beta = 1.25
gamma_1 = 0.01

def simulate_average_flood_damage(n_iterations=1000, m_years=30, alpha=alpha, beta=beta, gamma_scale=gamma_1, random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)

    total_damage = 0

    # Map severity levels to numeric factors
    severity_map = {"Low": 1, "Medium": 2, "High": 3, "Extreme": 4}

    for _ in range(n_iterations):
        # Step 1: Simulate flood frequency for each year
        flood_counts = simulate_flood_frequency(df_floods, n_years=m_years)

        # Step 2: Generate SLR and Precipitation values for each year
        slr_values = generate_random_slr(m_years)
        prcp_values = generate_random_precipitation(m_years)

        damage_sum = 0

        for year in range(m_years):
            floods_this_year = flood_counts[year]
            slr = slr_values[year]
            prcp = prcp_values[year]

            # Step 3: Assign severity for each flood
            severities = generate_random_severity(floods_this_year)

            # Step 4: Compute damage per flood and sum
            for sev in severities:
                severity_factor = severity_map[sev]
                damage = alpha * severity_factor * (1 + beta * slr) * (1 + gamma_scale * prcp)
                damage_sum += damage


        total_damage += damage_sum

    # Step 5: Return average damage over all iterations
    average_damage = total_damage / n_iterations
    return average_damage

In [ ]:
# Load dataset
df = pd.read_csv("/content/HazardMitigationAssistanceProjects.csv")

# Clean and filter for Louisiana
df['projectType_clean'] = df['projectType'].astype(str).str.lower()
df['dateApproved'] = pd.to_datetime(df['dateApproved'], errors='coerce')

# Filter for Louisiana and years 2020–2025
keep_cols = [
    'projectIdentifier', 'state', 'county', 'projectType', 'status',
    'dateApproved', 'projectAmount', 'federalShareObligated', 'benefitCostRatio',
    'numberOfProperties', 'netValueBenefits', 'costSharePercentage', 'projectType_clean'
]
df = df[(df['state'] == 'Louisiana') &
        (df['dateApproved'].dt.year >= 2020) &
        (df['dateApproved'].dt.year <= 2025)][keep_cols] #Assuming it is not implemented still started after 2020

# Disaster-related keywords
keywords = ['flood', 'floodwater', 'coastal', 'hurricane', 'stormwater', 'levee',
            'drainage', 'sea', 'level', 'elevation', 'floodproof', 'wind', 'tornado']
df = df[df['projectType_clean'].apply(lambda x: any(kw in x for kw in keywords))]

# Categorize mitigation types
def categorize(ptype):
    ptype = ptype.lower()
    if "acquisition" in ptype:
        return "Property Acquisition"
    elif "elevation" in ptype:
        return "Elevation"
    elif "relocation" in ptype:
        return "Relocation"
    elif "retrofit" in ptype:
        return "Structural Retrofitting"
    elif "stormwater" in ptype or "culvert" in ptype or "drainage" in ptype:
        return "Stormwater Management"
    elif "levee" in ptype or "floodwall" in ptype:
        return "Structural Flood Control"
    elif "natural" in ptype or "green" in ptype or "ecosystem" in ptype:
        return "Natural Infrastructure"
    elif "floodproof" in ptype:
        return "Floodproofing"
    else:
        return "Other / Misc"

df['mitigationCategory'] = df['projectType'].apply(categorize)

# Group by category
summary = df.groupby('mitigationCategory').agg({
    'projectAmount': 'sum',
    'benefitCostRatio': 'mean'
}).reset_index()

# Normalize BCR for weight and reduction factor
summary['weight'] = summary['benefitCostRatio'] / summary['benefitCostRatio'].sum()
summary['reduction_factor'] = summary['weight']

# Final DataFrame
final_df = summary[['mitigationCategory', 'projectAmount', 'benefitCostRatio','weight']]

# Display
final_df

,mitigationCategory,projectAmount,benefitCostRatio,weight
0,Elevation,1.139937e+09,2.139652,0.092675
1,Floodproofing,1.164000e+05,5.965000,0.258361
2,Other / Misc,2.584998e+08,3.874464,0.167814
3,Property Acquisition,1.373596e+08,1.706923,0.073932
4,Stormwater Management,4.592656e+08,1.973441,0.085475
5,Structural Flood Control,1.556066e+07,5.698571,0.246822
6,Structural Retrofitting,6.342577e+07,1.729767,0.074921


In [ ]:
# Define the function that randomly selects a mitigation category and applies its weight as a reduction factor
def get_random_mitigation_reduction(seed=None):
    if seed is not None:
        np.random.seed(seed)

    mitigation_data = {
        "mitigationCategory": [
            "Elevation", "Floodproofing", "Other / Misc", "Property Acquisition",
            "Stormwater Management", "Structural Flood Control", "Structural Retrofitting"
        ],
        "weight": [0.092675, 0.258361, 0.167814, 0.073932, 0.085475, 0.246822, 0.074921]
    }

    categories = mitigation_data["mitigationCategory"]
    weights = mitigation_data["weight"]

    selected_index = np.random.choice(len(categories))
    selected_category = categories[selected_index]
    reduction_factor = weights[selected_index]

    return selected_category, reduction_factor

# Modified simulation function to include mitigation reduction
def simulate_average_flood_damage_with_mitigation(n_iterations=1000, m_years=30, alpha=1000000, beta=1.25, gamma_scale=0.01, random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)

    total_damage = 0
    severity_map = {"Low": 1, "Medium": 2, "High": 3, "Extreme": 4}

    for _ in range(n_iterations):
        flood_counts = simulate_flood_frequency(df_floods, n_years=m_years)
        slr_values = generate_random_slr(m_years)
        prcp_values = generate_random_precipitation(m_years)

        damage_sum = 0

        for year in range(m_years):
            floods_this_year = flood_counts[year]
            slr = slr_values[year]
            prcp = prcp_values[year]

            severities = generate_random_severity(floods_this_year)
            _, reduction_factor = get_random_mitigation_reduction()  # Get random reduction factor

            for sev in severities:
                severity_factor = severity_map[sev]
                raw_damage = alpha * severity_factor * (1 + beta * slr) * (1 + gamma_scale * prcp)
                adjusted_damage = raw_damage * (1 - reduction_factor)
                damage_sum += adjusted_damage

        total_damage += damage_sum

    average_damage = total_damage / n_iterations
    return average_damage


In [ ]:
years = 30
avg_damage = simulate_average_flood_damage(n_iterations=100, m_years=years)
print(f"Estimated Total Damage over {years} years: ${avg_damage:,.2f}")

Estimated Total Damage over 30 years: $122,214,488,255.75


In [ ]:
avg_damage_mitigated = simulate_average_flood_damage_with_mitigation(n_iterations=100, m_years=years)
print(f"Estimated Total Damage over {years} years with Mitigation: ${avg_damage_mitigated:,.2f}")

Estimated Total Damage over 30 years with Mitigation: $107,510,921,426.14
